In [1]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt


In [5]:
full = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv")
full = full[['price', 'make_name', 'mileage']]
full

C:\Users\colec\AppData\Local\Temp\ipykernel_3620\1309838002.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv("Cleaned_data_v2/cleaned_car_data_v2.csv")


,price,make_name,mileage
0,200.0,Ford,150000.0
1,249.0,Buick,202158.0
2,347.0,Mazda,213512.0
3,395.0,Jeep,122610.0
4,400.0,Oldsmobile,111111.0
...,...,...,...
2528603,2175000.0,Nissan,570.0
2528604,2250000.0,Chevrolet,7852.0
2528605,2698500.0,Ford,39060.0
2528606,2738900.0,Ferrari,221.0


In [13]:
# ranges = [(0, 10000), (10000, 15000), (15000, 20000), (20000, 25000), (25000, 30000), 
#         (30000, 40000), (40000, 50000), (50000, 60000), (60000, 70000), (70000, max(full['price']) +1)]

ranges = [
    (0, full.price.quantile(.1)),
    (full.price.quantile(.1), full.price.quantile(.2)),
    (full.price.quantile(.2), full.price.quantile(.3)),
    (full.price.quantile(.3), full.price.quantile(.4)),
    (full.price.quantile(.4), full.price.quantile(.5)),
    (full.price.quantile(.5), full.price.quantile(.6)),
    (full.price.quantile(.6), full.price.quantile(.7)),
    (full.price.quantile(.7), full.price.quantile(.8)),
    (full.price.quantile(.8), full.price.quantile(.9)),
    (full.price.quantile(.9), full.price.quantile(1)+1),
]

# ranges = [
#     (0, full.price.quantile(.5)),
#     (full.price.quantile(.5), full.price.quantile(1)+1),
# ]

def findBucket(price):
    val = 0
    for startPrice, EndPrice in ranges:
        if price >= startPrice and price < EndPrice:
            return val
        val += 1

full['bucket'] = [0] * len(full) 
full['bucket'] = full.apply(lambda row: findBucket(row['price']), axis=1)

makes = {}
def encodeMake(make):
    if make not in makes.keys():
        makes[make] = len(makes)
    return makes[make]

full['make_name'] = full.apply(lambda row: encodeMake(row['make_name']), axis=1)

full

,price,make_name,mileage,bucket
0,200.0,0,150000.0,0
1,249.0,1,202158.0,0
2,347.0,2,213512.0,0
3,395.0,3,122610.0,0
4,400.0,4,111111.0,0
...,...,...,...,...
2528603,2175000.0,19,570.0,9
2528604,2250000.0,7,7852.0,9
2528605,2698500.0,0,39060.0,9
2528606,2738900.0,56,221.0,9


In [14]:
y = full.bucket
y = y.to_numpy()

x = full[['make_name', 'mileage']]
x = x.to_numpy()

for i in range(x.shape[1]):
    if np.sum(x[:, i]) == 0:
        continue
    x[:, i] /= np.sum(x[:, i])


In [15]:
# x = np.concatenate((x_train, x_test))
# y = np.concatenate((y_train, y_test))
# x = x.reshape((x.shape[0], -1))
# x = np.divide(x, 255.)
# 10 clusters
n_clusters = len(np.unique(y))
kmeans = KMeans(n_clusters=n_clusters, n_init=80)
# Train K-Means.
y_pred_kmeans = kmeans.fit_predict(x)
# Evaluate the K-Means clustering accuracy.
sklearn.metrics.accuracy_score(y, y_pred_kmeans) * 100

6.229474873131778

In [21]:

a = 0
for i in range(1, len(np.unique(y_pred_kmeans))):
    val = np.count_nonzero(y_pred_kmeans == i)
    a += val
    print(f"{i}: {val}")
print(f"0: {len(x) - a}")

a = 0
for i in range(1, len(np.unique(y))):
    val = np.count_nonzero(y == i)
    a += val
    print(f"{i}: {val}")
print(f"0: {len(x) - a}")

1: 2
0: 1211038
1: 607469
0: 603571


In [22]:
diff = y - y_pred_kmeans
diff = diff * diff
np.mean(diff)

0.5016077090764962